<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"
alt="Rubin Observatory logo, a graphical representation of turning stars into data."> <br><br>
**Description:** Generate delivered image quality plots for DP1 data. <br>
**Authors:** Original version for LSSTComCam: Keith Bechtol. Modified version for DP1:  Leanne Guy, <br>
**Last verified to run:** 2025-05-25 <br>
**LSST Science Pipelines version:** v29.0.1<br>
**Container size:** medium <br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

from lsst.daf.butler import Butler
import lsst.geom

# Set a standard figure size to use
from lsst.utils.plotting import publication_plots
from lsst.utils.plotting import get_multiband_plot_colors, get_multiband_plot_symbols, get_multiband_plot_linestyles

In [ ]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/raw/all', 
               'LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260' ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)

In [ ]:
# Applying the Rubin DP1 paper custom style sheet to all of the graphs to be created in this notebook
publication_plots.set_rubin_plotstyle()
bands_dict = publication_plots.get_band_dicts()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends
%matplotlib inline

In [ ]:
# Force to unpushed updated color scheme.
colors ={"u": "#48A8D4",
         "g": "#31DE1F",
         "r": "#B52626",
         "i": "#2915A4",
         "z": "#AD03EA",
         "y": "#2D0201",
}
bands = colors.keys()  # important to get the right order for plot legends

In [ ]:
visit_detector_table = butler.get("visit_detector_table", storageClass="DataFrame")

In [ ]:
# Extract and format data for analysis
df = visit_detector_table[['detectorId','visitId', 'band', 'psfSigma','seeing']].copy()
df["psfFwhm"]=df["psfSigma"]*2.355*0.2
df

In [ ]:
# Best delivered IQ
min_row = df.loc[df['psfFwhm'].idxmin()]
min_row

In [ ]:
# This is very low -- 0.135 arcsec seeing is not physical.
# How should these be treated in the analysis? 
rows = df[df['visitId'] == 2024112400119]
rows

In [ ]:
# This is clearly non-physical 
# Count number of visits with IQ <= 0.65 (or threshold)
t_low = 0.5
t_high = 0.6
df_iq_analysis = df[(df['psfFwhm'] >= t_low) & (df['psfFwhm'] < t_high)]
visit_detector_pairs = df_iq_analysis[['visitId', 'detectorId', 'psfFwhm']].drop_duplicates().values.tolist()
print(f"Visit/Dector pairs with IQ in {t_low}, {t_high}\nID (visitId, detectorId, psfFwhm): {visit_detector_pairs}")

In [ ]:
# This is clearly non-physical 
# Current analysis -- Put a lower cut on IQ at 0.6  to exclude non-physical values 
# -- based on SITCOMTN report of 0.65 bing best IQ 

# remove artificial cut for analysis
#df = df[df['psfFwhm'] >= 0.6]

In [ ]:
min_row = df.loc[df['psfFwhm'].idxmin()]
print(f"Best delivered image quality {min_row}")

In [ ]:
# Compute all band summary statistics
quantiles = [25, 50, 75]

iq_summary_all = df['psfFwhm'].quantile([q/100 for q in quantiles]).apply(
    lambda x: round(x, 2)).to_frame().T
iq_summary_all['mean'] = round(df['psfFwhm'].mean(), 2)
iq_summary_all.index = ['all']
iq_summary_all.index.name = 'band'

# Compute summary statistics by band 
iq_summary_band = df.groupby('band')['psfFwhm'].quantile(
    [q / 100 for q in quantiles]).apply(
    lambda x: round(x, 2)).unstack()
iq_summary_band['mean'] = df.groupby('band')['psfFwhm'].mean().round(2)
iq_summary_band.index.name = 'band'

# Concatenate and sort and set type 
iq_summary = pd.concat([iq_summary_band, iq_summary_all])
iq_summary = iq_summary.astype(float)
band_order =  list(bands) +  ['all'] 
iq_summary.index = pd.CategoricalIndex(
    iq_summary.index, categories=band_order, 
    ordered=True, name='band')
iq_summary = iq_summary.sort_index()
iq_summary

## Plots of the Image quality 

In [ ]:
# Extract data from visit table for plotting
data = {}
for band in bands:
    data[band] = np.array(df[df['band'].str.contains(band)]['psfFwhm']) 

In [ ]:
# IQ histogram - not for inclusion in the paper but interesting
plt.figure()

for label, d in data.items():
    plt.hist(d, bins=20, alpha=0.5,
             linestyle='-',
             color = colors[label],
             label=label)

# Customize plot
plt.xlabel('PSF FWHM [arcsec]')
plt.ylabel('Fraction of Sensors')
plt.grid(True)
plt.legend()
# plt.xlim(0, 0.9)  
# plt.ylim(0, 500) 

plt.savefig("../figures/image_quality_histo.pdf", 
            bbox_inches='tight',  # Trim whitespace around the figure
            transparent=True,     # Transparent background
            format='png')         # Explicit format specification
plt.show()

In [ ]:
# IQ ECDF
plt.figure()

for label, d in data.items():
    ecdf = ECDF(d)
    plt.plot(ecdf.x, ecdf.y, 
             linestyle='-',
             color = colors[label],
             label=label)

# Customize plot
plt.xlabel('PSF FWHM [arcsec]')
plt.ylabel('Fraction of Sensors')
plt.grid(True)
plt.xlim(0.4, 2.7)
plt.legend()
plt.savefig("../figures/image_quality_ecdf.pdf", 
            bbox_inches='tight',  # Trim whitespace around the figure
            transparent=True,     # Transparent background
            format='png')         # Explicit format specification
plt.show()

In [ ]:
# Investigate the variation by field -- expect to be minimal
# TODO

In [ ]:
plt.close()

## Write out latex tables

In [ ]:
# Convert to latex format
# Insert blank colum for nice spacing in table 
data_latex = iq_summary.drop('mean', axis=1)
data_latex.insert(0, ' ', ' ')
data_latex = data_latex.to_latex(index=True, 
                                       bold_rows=False, float_format='%.2f'
                            )
data_latex = data_latex.split("midrule\n", 1)[-1]  # Keeps the part after 'midrule'
data_latex = data_latex.split("\\bottomrule", 1)[0]  # Keeps the part before 'bottomrule'
data_latex

In [ ]:
# Write to latex table
with open("../tables/image_quality_summary.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\setlength{\tabcolsep}{14pt} 
\begin{deluxetable*}{ccccc}
\tablecaption{Image quality expressed in terms of PSF FWHM in arcseconds per band and for all bands.
\label{tab:image_quality} }
\tablehead{
  \colhead{\textbf{Band}} && \multicolumn{3}{c}{\textbf{Quantile (\%)}} \\
  \cline{3-5}
   & & 25& 50& 75 
}
\startdata
""")
    f.write(data_latex)
    f.write(r"""\enddata
\end{deluxetable*}
""")
f.close()